In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import sklearn.decomposition as sk_decomp

In [2]:
user_path = "/Users/alexseto/Documents/GitHub/RecipeBandit"

In [3]:
import pandas as pd

train_data = pd.read_csv(user_path + '/data/interactions_train.csv')
val_data = pd.read_csv(user_path + '/data/interactions_validation.csv')
test_data = pd.read_csv(user_path + '/data/interactions_test.csv')

NUM_USERS = 25076
NUM_RECIPES = 178265

def interactions_to_matrix(data):
    result = np.zeros([NUM_USERS, NUM_RECIPES])
    for index, row in data.iterrows():
      result[int(row['u']), int(row['i'])] = float(row['rating'])
    return result

user_recipes = interactions_to_matrix(train_data)

In [25]:
RECIPE_LIMIT = 1000

def crop_matrix(user_recipes, RECIPE_LIMIT):
    cropped = user_recipes[:, :RECIPE_LIMIT]
    nonzero_users = list(np.nonzero(np.sum(cropped, axis=1) > 0.0)[0])
    number_nonzero_users = len(nonzero_users)
    return cropped[nonzero_users, :]

In [39]:


#TAKEN FROM SOME RANDOM SITE

def select_n_components(var_ratio, goal_var: float) -> int:
    total_variance = 0.0
    
    n_components = 0
    
    for explained_variance in var_ratio:
        
        total_variance += explained_variance
        
        n_components += 1
        
        if total_variance >= goal_var:
            break

    return n_components

def produce_optimal_svd_ncomps(X, goal_var = .8):
    tsvd = sk_decomp.TruncatedSVD(n_components=cropped.shape[1]-1)
    cropped_tsvd = tsvd.fit(cropped)
    tsvd_var_ratios = cropped_tsvd.explained_variance_ratio_
    n_components = select_n_components(tsvd_var_ratios, goal_var)
    return n_components

In [68]:
n_comps = produce_optimal_svd_ncomps(cropped, .5)
n_comps

52

In [51]:
svd = sk_decomp.TruncatedSVD(n_comps)
X_t  = svd.fit_transform(cropped)

In [52]:
svd.inverse_transform(X_t)

array([[ 3.98204521e+00, -1.09369583e-03, -5.50069000e-03, ...,
         2.28266052e-03,  1.07487132e-03, -4.28037135e-03],
       [ 4.97755652e+00, -1.36711978e-03, -6.87586250e-03, ...,
         2.85332564e-03,  1.34358915e-03, -5.35046419e-03],
       [-1.09369583e-03,  8.65455991e-03,  3.46558230e-03, ...,
        -4.70155091e-04,  1.51898259e-04,  1.65356399e-03],
       ...,
       [ 2.83098832e-05, -1.66203576e-04,  7.27353705e-04, ...,
         9.54807429e-04,  3.18911347e-03, -1.35296833e-03],
       [ 2.83098832e-05, -1.66203576e-04,  7.27353705e-04, ...,
         9.54807429e-04,  3.18911347e-03, -1.35296833e-03],
       [ 2.83098832e-05, -1.66203576e-04,  7.27353705e-04, ...,
         9.54807429e-04,  3.18911347e-03, -1.35296833e-03]])

In [53]:
n_comps

514

In [55]:
def shrink_dimensions_svd(X, goal_var=.95):
    n_comps = produce_optimal_svd_ncomps(X, goal_var)
    svd = sk_decomp.TruncatedSVD(n_comps)
    X_t  = svd.fit_transform(cropped)
    return X_t, svd

In [56]:
X_t, svd = shrink_dimensions_svd(X, goal_var = .95)

In [59]:
X_t.shape[1]

514

ALL FUNCS

In [60]:
NUM_USERS = 25076
NUM_RECIPES = 178265

def interactions_to_matrix(data):
    result = np.zeros([NUM_USERS, NUM_RECIPES])
    for index, row in data.iterrows():
      result[int(row['u']), int(row['i'])] = float(row['rating'])
    return result

RECIPE_LIMIT = 1000

def crop_matrix(user_recipes, RECIPE_LIMIT):
    cropped = user_recipes[:, :RECIPE_LIMIT]
    nonzero_users = list(np.nonzero(np.sum(cropped, axis=1) > 0.0)[0])
    number_nonzero_users = len(nonzero_users)
    return cropped[nonzero_users, :]

def select_n_components(var_ratio, goal_var: float) -> int:
    total_variance = 0.0
    
    n_components = 0
    
    for explained_variance in var_ratio:
        
        total_variance += explained_variance
        
        n_components += 1
        
        if total_variance >= goal_var:
            break

    return n_components

def produce_optimal_svd_ncomps(X, goal_var = .8):
    tsvd = sk_decomp.TruncatedSVD(n_components=cropped.shape[1]-1)
    cropped_tsvd = tsvd.fit(cropped)
    tsvd_var_ratios = cropped_tsvd.explained_variance_ratio_
    n_components = select_n_components(tsvd_var_ratios, goal_var)
    return n_components

def shrink_dimensions_svd(X, goal_var=.95):
    n_comps = produce_optimal_svd_ncomps(X, goal_var)
    svd = sk_decomp.TruncatedSVD(n_comps)
    X_t  = svd.fit_transform(cropped)
    return X_t, svd

In [64]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools
import pandas as pd
import ast
import os
from absl import app
from absl import flags

import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import

from tf_agents.bandits.agents import lin_ucb_agent
from tf_agents.bandits.agents.examples.v2 import trainer
from tf_agents.bandits.environments import environment_utilities
from tf_agents.bandits.environments import movielens_per_arm_py_environment
from tf_agents.bandits.environments import movielens_py_environment
from tf_agents.bandits.metrics import tf_metrics as tf_bandit_metrics
from tf_agents.bandits.networks import global_and_arm_feature_network
from tf_agents.environments import tf_py_environment


import random
from typing import Optional, Text
import gin
import numpy as np

from tf_agents.bandits.environments import bandit_py_environment
from tf_agents.bandits.environments import dataset_utilities
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts

import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import
from tf_agents.drivers import dynamic_step_driver
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.policies import policy_saver
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.bandits.specs import utils as bandit_spec_utils

import sklearn.decomposition as sk_decomp

import pandas as pd

train_data = pd.read_csv('../data/interactions_train.csv')
val_data = pd.read_csv('../data/interactions_validation.csv')
test_data = pd.read_csv('../data/interactions_test.csv')

NUM_USERS = 25076
NUM_RECIPES = 178265

def interactions_to_matrix(data):
    result = np.zeros([NUM_USERS, NUM_RECIPES])
    for index, row in data.iterrows():
      result[int(row['u']), int(row['i'])] = float(row['rating'])
    return result

user_recipes = interactions_to_matrix(train_data)

NUM_USERS = 25076
NUM_RECIPES = 178265

def interactions_to_matrix(data):
    result = np.zeros([NUM_USERS, NUM_RECIPES])
    for index, row in data.iterrows():
      result[int(row['u']), int(row['i'])] = float(row['rating'])
    return result

RECIPE_LIMIT = 1000

def crop_matrix(user_recipes, RECIPE_LIMIT):
    cropped = user_recipes[:, :RECIPE_LIMIT]
    nonzero_users = list(np.nonzero(np.sum(cropped, axis=1) > 0.0)[0])
    number_nonzero_users = len(nonzero_users)
    return cropped[nonzero_users, :]

def select_n_components(var_ratio, goal_var: float) -> int:
    total_variance = 0.0
    
    n_components = 0
    
    for explained_variance in var_ratio:
        
        total_variance += explained_variance
        
        n_components += 1
        
        if total_variance >= goal_var:
            break

    return n_components

def produce_optimal_svd_ncomps(X, goal_var = .8):
    tsvd = sk_decomp.TruncatedSVD(n_components=X.shape[1]-1)
    cropped_tsvd = tsvd.fit(X)
    tsvd_var_ratios = cropped_tsvd.explained_variance_ratio_
    n_components = select_n_components(tsvd_var_ratios, goal_var)
    return n_components

def shrink_dimensions_svd(X, goal_var=.95):
    n_comps = produce_optimal_svd_ncomps(X, goal_var)
    svd = sk_decomp.TruncatedSVD(n_comps)
    X_t  = svd.fit_transform(X)
    return X_t, svd, n_comps

In [65]:
class RecipePyEnvironment(bandit_py_environment.BanditPyEnvironment):
  """Implements the Recipe aBandit environment.
  This environment computes a low-rank matrix factorization (using SVD) of the
  data matrix A, such that: A ~= U * V.
  The reward of recommending item `j` to user `i` is provided as A_{ij}.
  """

  def __init__(self,
               user_recipes: np.array,
               batch_size: int = 1,
               num_recipes: int = 50,
               name: str = 'recipe'):
    """Initializes the MovieLens Bandit environment.
    Args:
      data_dir: (string) Directory where the data lies (in text form).
      rank_k : (int) Which rank to use in the matrix factorization.
      batch_size: (int) Number of observations generated per call.
      num_recipes: (int) Only the first `num_recipes` movies will be used by the
        environment. The rest is cut out from the data.
      name: The name of this environment instance.
    """
    self._num_actions = num_recipes

    self._data_matrix = crop_matrix(user_recipes, num_recipes)
    self._effective_num_users = self._data_matrix.shape[0]
    
    self._batch_size = batch_size
    
    self._data_matrix_svd, svd, optimal_rank_k = shrink_dimensions_svd(self._data_matrix, goal_var = .95)
    
    self._context_dim = optimal_rank_k
    
    self._approx_ratings_matrix = svd.inverse_transform(self._data_matrix_svd)

    self._current_users = np.zeros(batch_size)
    self._previous_users = np.zeros(batch_size)
####################################################################################################
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(),
        dtype=np.int32,
        minimum=0,
        maximum=self._num_actions - 1,
        name='action')
    
    observation_spec = array_spec.ArraySpec(
        shape=(self._context_dim,), dtype=np.float64, name='observation')

    self._time_step_spec = ts.time_step_spec(observation_spec)
    
    self._observation = np.zeros((self._batch_size, self._context_dim))

####################################################################################################   
    
    self._optimal_action_table = np.argmax(
        self._approx_ratings_matrix, axis=1)
    self._optimal_reward_table = np.max(
        self._approx_ratings_matrix, axis=1)

    super(RecipePyEnvironment, self).__init__(
        observation_spec, self._action_spec, name=name)

  @property
  def batch_size(self):
    return self._batch_size

  @property
  def batched(self):
    return True

  def _observe(self):
    """Returns the u vectors of a random sample of users."""
    sampled_users = random.sample(
        range(self._effective_num_users), self._batch_size)
    self._previous_users = self._current_users
    self._current_users = sampled_users
    batched_observations = self._data_matrix_svd[sampled_users]
    return batched_observations

  def _apply_action(self, action):
    """Computes the reward for the input actions."""
    rewards = []
    for i, j in zip(self._current_users, action):
      rewards.append(self._approx_ratings_matrix[i, j])
    return np.array(rewards)

  def compute_optimal_action(self):
    return self._optimal_action_table[self._previous_users]

  def compute_optimal_reward(self):
    return self._optimal_reward_table[self._previous_users]